In [1]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from openavalancheproject.prep_ml import PrepML

In [3]:
interpolate = 1 
data_root = '/media/scottcha/E1/Data/OAPMLData/'

pml = PrepML(data_root, interpolate, date_start='2015-11-01', date_end='2019-04-30', date_train_test_cutoff='2018-11-01')

pml.regions = {'Colorado': ['Grand Mesa Zone', 'Sangre de Cristo Range', 'Steamboat Zone', 'Front Range Zone',
           'Vail Summit Zone', 'Sawatch Zone', 'Aspen Zone', 
           'North San Juan Mountains', 'South San Juan Mountains', 'Gunnison Zone']}

train, test = pml.prep_labels()

Grand Mesa Zone
Sangre de Cristo Range
Steamboat Zone
Front Range Zone
Vail Summit Zone
Sawatch Zone
Aspen Zone
North San Juan Mountains
South San Juan Mountains
Gunnison Zone


In [4]:
list(train.columns)

['UnifiedRegion',
 'latitude',
 'longitude',
 'UnifiedRegionleft',
 'Unnamed: 0',
 'BottomLineSummary',
 'Cornices_Likelihood',
 'Cornices_MaximumSize',
 'Cornices_MinimumSize',
 'Cornices_OctagonAboveTreelineEast',
 'Cornices_OctagonAboveTreelineNorth',
 'Cornices_OctagonAboveTreelineNorthEast',
 'Cornices_OctagonAboveTreelineNorthWest',
 'Cornices_OctagonAboveTreelineSouth',
 'Cornices_OctagonAboveTreelineSouthEast',
 'Cornices_OctagonAboveTreelineSouthWest',
 'Cornices_OctagonAboveTreelineWest',
 'Cornices_OctagonBelowTreelineEast',
 'Cornices_OctagonBelowTreelineNorth',
 'Cornices_OctagonBelowTreelineNorthEast',
 'Cornices_OctagonBelowTreelineNorthWest',
 'Cornices_OctagonBelowTreelineSouth',
 'Cornices_OctagonBelowTreelineSouthEast',
 'Cornices_OctagonBelowTreelineSouthWest',
 'Cornices_OctagonBelowTreelineWest',
 'Cornices_OctagonNearTreelineEast',
 'Cornices_OctagonNearTreelineNorth',
 'Cornices_OctagonNearTreelineNorthEast',
 'Cornices_OctagonNearTreelineNorthWest',
 'Cornices_

In [12]:
interesting_labels =  ['Day1DangerAboveTreeline',
                         'Day1DangerBelowTreeline',
                         'Day1DangerNearTreeline',
                         #'Cornices_Likelihood', #these are basically absent from the CAIC dataset, raised with CAIC
                         #'Cornices_MaximumSize',
                         #'Cornices_MinimumSize',                         
                         'DeepPersistentSlab_Likelihood',
                         'DeepPersistentSlab_MaximumSize',
                         'DeepPersistentSlab_MinimumSize',
                         'Glide_Likelihood',
                         'Glide_MaximumSize',
                         'Glide_MinimumSize',
                         'LooseDry_Likelihood',
                         'LooseDry_MaximumSize',
                         'LooseDry_MinimumSize',
                         'LooseWet_Likelihood',
                         'LooseWet_MaximumSize',
                         'LooseWet_MinimumSize',
                         'PersistentSlab_Likelihood',
                         'PersistentSlab_MaximumSize',
                         'PersistentSlab_MinimumSize',
                         'StormSlabs_Likelihood',
                         'StormSlabs_MaximumSize',
                         'StormSlabs_MinimumSize',
                         'WetSlabs_Likelihood',
                         'WetSlabs_MaximumSize',
                         'WetSlabs_MinimumSize',
                         'WindSlab_Likelihood',
                         'WindSlab_MaximumSize',
                         'WindSlab_MinimumSize',
                         'WindSlab_OctagonAboveTreelineEast',
                         'WindSlab_OctagonAboveTreelineNorth',
                         'WindSlab_OctagonAboveTreelineNorthEast',
                         'WindSlab_OctagonAboveTreelineNorthWest',
                         'WindSlab_OctagonAboveTreelineSouth',
                         'WindSlab_OctagonAboveTreelineSouthEast',
                         'WindSlab_OctagonAboveTreelineSouthWest',
                         'WindSlab_OctagonAboveTreelineWest',
                         'WindSlab_OctagonBelowTreelineEast',
                         'WindSlab_OctagonBelowTreelineNorth',
                         'WindSlab_OctagonBelowTreelineNorthEast',
                         'WindSlab_OctagonBelowTreelineNorthWest',
                         'WindSlab_OctagonBelowTreelineSouth',
                         'WindSlab_OctagonBelowTreelineSouthEast',
                         'WindSlab_OctagonBelowTreelineSouthWest',
                         'WindSlab_OctagonBelowTreelineWest',
                         'WindSlab_OctagonNearTreelineEast',
                         'WindSlab_OctagonNearTreelineNorth',
                         'WindSlab_OctagonNearTreelineNorthEast',
                         'WindSlab_OctagonNearTreelineNorthWest',
                         'WindSlab_OctagonNearTreelineSouth',
                         'WindSlab_OctagonNearTreelineSouthEast',
                         'WindSlab_OctagonNearTreelineSouthWest',
                         'WindSlab_OctagonNearTreelineWest',]

In [6]:
len(interesting_labels)

51

In [14]:
for i_f in interesting_labels[:3]:
    #get the possible labels
    tmp_labels = list(train[i_f].unique())
    tmp_test_labels = list(test[i_f].unique())
    
    special = ['Day1DangerAboveTreeline', 'Day1DangerBelowTreeline','Day1DangerNearTreeline']
        
    if i_f in special:
        #in this case only use these three lables (removes extreme and no-data)
        tmp_labels = set(tmp_labels).intersection(['Low', 'Moderate', 'Considerable', 'High'])
        tmp_test_labels = set(tmp_test_labels).intersection(['Low', 'Moderate', 'Considerable', 'High'])
        
    if len(tmp_labels)==1:
        print('Skipping ' + i_f + ' which only has one label: ' + tmp_labels[0])
        continue
    #if len(tmp_labels) != len(tmp_test_labels):
    #    print('Skipping ' + i_f + ' which doesnt have the same train and test labels')
    #    continue
    oversample = {}
    
    print('Processing ' + i_f)
    #not oversampling anything for this pass
    for l in tmp_labels:
        oversample[l] = False
    
    train2, test2 = pml.generate_train_test_local(train, 
                                                  test, 
                                                  num_train_files=1, 
                                                  num_test_files=1, 
                                                  num_train_rows_per_file=5000, 
                                                  num_test_rows_per_file=1000, 
                                                  batch_size=1000,
                                                  file_label='co_' + i_f + '_small', 
                                                  y_column=i_f,
                                                  label_values=tmp_labels,
                                                  oversample=oversample,
                                                  temp_destination_path= '/media/scottcha/4A1E51311E5116F3/Temp/', 
                                                  n_jobs=8)   
   

Processing Day1DangerAboveTreeline
Getting a batch for label Day1DangerAboveTreeline
    on label: Low with samplesize: 1
    len: 21834
    on label: Moderate with samplesize: 1
    len: 46808
    on label: Considerable with samplesize: 1
    len: 12791
    on label: High with samplesize: 1
    len: 1740
have n_jobs -1
On 0 of 5000
Getting a batch for label Day1DangerAboveTreeline
    on label: Low with samplesize: 250
    len: 21834
    on label: Considerable with samplesize: 250
    len: 12791
    on label: High with samplesize: 250
    len: 1740
    on label: Moderate with samplesize: 250
    len: 46808
have n_jobs 8
start: 0 end: 1000
(1000, 978, 180)
1000
On 1000 of 5000
Getting a batch for label Day1DangerAboveTreeline
    on label: Low with samplesize: 250
    len: 21584
    on label: Considerable with samplesize: 250
    len: 12541
    on label: High with samplesize: 250
    len: 1490
    on label: Moderate with samplesize: 250
    len: 46558
have n_jobs 8
start: 1000 end: 200

    on label: High with samplesize: 250
    len: 661
    on label: Moderate with samplesize: 250
    len: 44670
have n_jobs 8
start: 4000 end: 5000
(1000, 978, 180)
1000
On 0 of 1000
Getting a batch for label Day1DangerNearTreeline
    on label: Low with samplesize: 250
    len: 6738
    on label: Considerable with samplesize: 250
    len: 7691
    on label: High with samplesize: 250
    len: 1620
    on label: Moderate with samplesize: 250
    len: 18492
have n_jobs 8
start: 0 end: 1000
(1000, 978, 180)
1000
['/media/scottcha/E1/Data/OAPMLData//5.MLData/Xtrain_batch_0_co_Day1DangerNearTreeline_small_on_disk.npy', '/media/scottcha/E1/Data/OAPMLData//5.MLData/Xtest_batch_0_co_Day1DangerNearTreeline_small_on_disk.npy']
On file 0 of 2
On file 1 of 2
